<a href="https://colab.research.google.com/github/Maucho6340/Telecom_X_LATAM_Predict/blob/main/Challenge_predict_evasion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'> CHALLENGE TELECOM X:
**Análisis de evasión de clientes Parte-2**

## MISIÓN



Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

## OBJETIVOS DEL DESAFÍO



- Preparar los datos para el modelado (tratamiento, codificación, normalización).

- Realizar análisis de correlación y selección de variables.

- Entrenar dos o más modelos de clasificación.

- Evaluar el rendimiento de los modelos con métricas.

- Interpretar los resultados, incluyendo la importancia de las variables.

- Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.

## DESARROLLO

### Cargar archivo

In [13]:
import pandas as pd

# Cargar archivo CSV
df_final = pd.read_csv("/content/df_predict_churn.csv")

# Vista rápida de los primeros registros
df_final.head(5)

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [11]:
import pandas as pd
import ast

# 1. Cargar archivo CSV
df_raw = pd.read_csv("/content/df_predict_churn.csv")

# 2. Convertir strings tipo dict a dict real
dict_cols = ["customer", "phone", "internet", "account"]
for col in dict_cols:
    df_raw[col] = df_raw[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# 3. Construir DataFrame plano
df_final = pd.DataFrame({
    "ID_Cliente": df_raw["customerID"],
    "Cancelación": df_raw["Churn"].map({"Yes": 1, "No": 0}),
    "Género": df_raw["customer"].apply(lambda x: x.get("gender")),
    "Mayor_65": df_raw["customer"].apply(lambda x: x.get("SeniorCitizen")),
    "Tiene_Pareja": df_raw["customer"].apply(lambda x: x.get("Partner")),
    "Tiene_Dependientes": df_raw["customer"].apply(lambda x: x.get("Dependents")),
    "Meses_Contrato": df_raw["customer"].apply(lambda x: round(x.get("tenure", 0) / 12, 6)),
    "Servicio_Teléfono": df_raw["phone"].apply(lambda x: 1 if x.get("PhoneService") == "Yes" else 0),
    "Líneas_Múltiples": df_raw["phone"].apply(lambda x: 1 if x.get("MultipleLines") == "Yes" else 0),
    "Proveedor_Internet": df_raw["internet"].apply(lambda x: x.get("InternetService")),
    "Seguridad_Online": df_raw["internet"].apply(lambda x: x.get("OnlineSecurity")),
    "Respaldo_Online": df_raw["internet"].apply(lambda x: x.get("OnlineBackup")),
    "Protección_Dispositivo": df_raw["internet"].apply(lambda x: x.get("DeviceProtection")),
    "Soporte_Técnico": df_raw["internet"].apply(lambda x: x.get("TechSupport")),
    "Servicio_TV": df_raw["internet"].apply(lambda x: x.get("StreamingTV")),
    "Servicio_Cine": df_raw["internet"].apply(lambda x: x.get("StreamingMovies")),
    "Tipo_Contrato": df_raw["account"].apply(lambda x: x.get("Contract")),
    "Factura_Electrónica": df_raw["account"].apply(lambda x: 1 if x.get("PaperlessBilling") == "Yes" else 0),
    "Método_Pago": df_raw["account"].apply(lambda x: x.get("PaymentMethod")),
    "Monto_Mensual": df_raw["account"].apply(lambda x: float(x.get("MonthlyCharges", 0))),
    "Monto_Total": df_raw["account"].apply(lambda x: float(x.get("TotalCharges", 0)) if x.get("TotalCharges") not in ["", None] else 0.0),
})

# 4. Agregar columnas derivadas
df_final["Cuentas_diarias"] = df_final["Monto_Total"] / df_final["Monto_Mensual"]
df_final["Cancelación_binaria"] = df_final["Cancelación"]

# 5. Vista rápida
df_final.head(5)

,ID_Cliente,Cancelación,Género,Mayor_65,Tiene_Pareja,Tiene_Dependientes,Meses_Contrato,Servicio_Teléfono,Líneas_Múltiples,Proveedor_Internet,...,Soporte_Técnico,Servicio_TV,Servicio_Cine,Tipo_Contrato,Factura_Electrónica,Método_Pago,Monto_Mensual,Monto_Total,Cuentas_diarias,Cancelación_binaria
0,0002-ORFBO,0.0,Female,0,Yes,Yes,0.750000,1,0,DSL,...,Yes,Yes,No,One year,1,Mailed check,0.0,0.0,NaN,0.0
1,0003-MKNFE,0.0,Male,0,No,No,0.750000,1,1,DSL,...,No,No,Yes,Month-to-month,0,Mailed check,0.0,0.0,NaN,0.0
2,0004-TLHLJ,1.0,Male,0,No,No,0.333333,1,0,Fiber optic,...,No,No,No,Month-to-month,1,Electronic check,0.0,0.0,NaN,1.0
3,0011-IGKFF,1.0,Male,1,Yes,No,1.083333,1,0,Fiber optic,...,No,Yes,Yes,Month-to-month,1,Electronic check,0.0,0.0,NaN,1.0
4,0013-EXCHZ,1.0,Female,1,Yes,No,0.250000,1,0,Fiber optic,...,Yes,Yes,No,Month-to-month,1,Mailed check,0.0,0.0,NaN,1.0


In [12]:
df_final["Cuentas_diarias"] = df_final.apply(
    lambda x: x["Monto_Total"] / x["Monto_Mensual"] if x["Monto_Mensual"] != 0 else 0,
    axis=1
)
df_final.head(5)

,ID_Cliente,Cancelación,Género,Mayor_65,Tiene_Pareja,Tiene_Dependientes,Meses_Contrato,Servicio_Teléfono,Líneas_Múltiples,Proveedor_Internet,...,Soporte_Técnico,Servicio_TV,Servicio_Cine,Tipo_Contrato,Factura_Electrónica,Método_Pago,Monto_Mensual,Monto_Total,Cuentas_diarias,Cancelación_binaria
0,0002-ORFBO,0.0,Female,0,Yes,Yes,0.750000,1,0,DSL,...,Yes,Yes,No,One year,1,Mailed check,0.0,0.0,0,0.0
1,0003-MKNFE,0.0,Male,0,No,No,0.750000,1,1,DSL,...,No,No,Yes,Month-to-month,0,Mailed check,0.0,0.0,0,0.0
2,0004-TLHLJ,1.0,Male,0,No,No,0.333333,1,0,Fiber optic,...,No,No,No,Month-to-month,1,Electronic check,0.0,0.0,0,1.0
3,0011-IGKFF,1.0,Male,1,Yes,No,1.083333,1,0,Fiber optic,...,No,Yes,Yes,Month-to-month,1,Electronic check,0.0,0.0,0,1.0
4,0013-EXCHZ,1.0,Female,1,Yes,No,0.250000,1,0,Fiber optic,...,Yes,Yes,No,Month-to-month,1,Mailed check,0.0,0.0,0,1.0
